In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Model, Sequential
from keras import layers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
df = pd.read_csv('/kaggle/input/poem-classification-nlp/Poem_classification - train_data.csv')

In [3]:
df.head()

,Genre,Poem
0,Music,NaN
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...


In [4]:
df.dropna(inplace= True)

In [5]:
df.head()

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."


In [6]:
df['Genre'].unique()

array(['Music', 'Death', 'Affection', 'Environment'], dtype=object)

In [7]:
df['Poem'].apply(lambda x: x.strip())

1      In the thick brushthey spend the hottest part ...
2      Storms are generous.                          ...
3      —After Ana Mendieta Did you carry around the m...
4      for Aja Sherrard at 20The portent may itself b...
5      for Bob Marley, Bavaria, November 1980 Here is...
                             ...                        
836    Why make so much of fragmentary blue In here a...
837    Woman, I wish I didn't know your name. What co...
838    Yonder to the kiosk, beside the creek, Paddle ...
839    You come to fetch me from my work to-night Whe...
840    You see them through water and glass, (both li...
Name: Poem, Length: 837, dtype: object

In [8]:
label_dummies = pd.get_dummies(df['Genre'])

In [9]:
categories = label_dummies.columns

In [10]:
labels = pd.get_dummies(df['Genre']).values.astype(np.float32)
poems_df = df['Poem'].values.astype(str)

In [11]:
labels = labels.argmax(-1)

In [12]:
len(labels)

837

In [13]:
poems_df[0]

'\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 In the thick brushthey spend the hottest part of the day,\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 soaking their hoovesin the trickle of mountain water\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 the ravine hoardson behalf of the oleander.\xa0 \xa0 \xa0 \xa0 \xa0 \xa0'

In [14]:
poems = []
for poem in poems_df:
    poems.append(poem.replace('\xa0', '').strip())
poems = np.array(poems)

In [15]:
poems.shape

(837,)

In [16]:
poems_len = [len(poem) for poem in poems]
max_len = max(poems_len)

In [17]:
max_len

466

In [18]:
v = layers.TextVectorization(output_sequence_length= max_len)

In [19]:
v.adapt(poems)

In [20]:
vocab_size = len(v.get_vocabulary(False))
vocab_size

9664

In [21]:
vocab_model = Sequential()
vocab_model.add(layers.Input((1,) , dtype= tf.string))
vocab_model.add(v)

In [22]:
vocab_vector = vocab_model.predict(poems)

27/27 [==============================] - 0s 2ms/step


In [23]:
vocab_vector[0], labels[0]

(array([   8,    2,  663, 8850, 1927,    2, 2450,  601,    5,    2,   70,
        4475,   50, 6866,    2,  914,    5,  311,  115,    2, 2085, 6896,
        9156,    5,    2, 5744,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [24]:
vocab_vector.shape, len(labels)

((837, 466), 837)

In [25]:
labels = tf.convert_to_tensor(labels, dtype= tf.int64)

In [26]:
labels[0]

<tf.Tensor: shape=(), dtype=int64, numpy=3>

In [27]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads= num_heads, key_dim= embed_dim)
        self.ffn = Sequential(
            [layers.Dense(ff_dim, activation= 'relu'), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon= 1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon= 1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training= training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training= training)
        return self.layernorm2(out1 + ffn_output)

In [28]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim= vocab_size, output_dim= embed_dim)
        self.pos_emb = layers.Embedding(input_dim= maxlen, output_dim= embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start= 0, limit= maxlen, delta= 1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [29]:
vocab_size, max_len

(9664, 466)

In [30]:
embed_dim = 32
num_heads = 2
ff_dim = 32

def transformer(maxlen= max_len,
                vocab_size= vocab_size,
                embed_dim= 32,
                num_heads= 2,
                ff_dim= 32):
    
    inputs = layers.Input(shape=(maxlen,))
    
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    
    x = embedding_layer(inputs)    
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(20, activation= 'relu')(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(len(np.unique(labels)), activation= 'softmax')(x)

    model = Model(inputs, outputs)
    return model

In [31]:
transformer_model = transformer()

In [32]:
transformer_model.compile('adam', loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])

In [33]:
transformer_model.fit(vocab_vector, labels,
           epochs= 100,
           batch_size= 32)

Epoch 1/100
27/27 [==============================] - 13s 203ms/step - loss: 1.4137 - accuracy: 0.2748
Epoch 2/100
27/27 [==============================] - 4s 160ms/step - loss: 1.3813 - accuracy: 0.2736
Epoch 3/100
27/27 [==============================] - 5s 167ms/step - loss: 1.3807 - accuracy: 0.3011
Epoch 4/100
27/27 [==============================] - 3s 104ms/step - loss: 1.3782 - accuracy: 0.2879
Epoch 5/100
27/27 [==============================] - 3s 122ms/step - loss: 1.3692 - accuracy: 0.2593
Epoch 6/100
27/27 [==============================] - 3s 91ms/step - loss: 1.3498 - accuracy: 0.3106
Epoch 7/100
27/27 [==============================] - 2s 74ms/step - loss: 1.3467 - accuracy: 0.3297
Epoch 8/100
27/27 [==============================] - 2s 80ms/step - loss: 1.3395 - accuracy: 0.3309
Epoch 9/100
27/27 [==============================] - 2s 76ms/step - loss: 1.3206 - accuracy: 0.3907
Epoch 10/100
27/27 [==============================] - 1s 50ms/step - loss: 1.2892 - accuracy: 

In [34]:
def predicted_label(text):
    text_vocab = vocab_model.predict([text])
    pred = transformer_model.predict(text_vocab)
    pred_idx = pred.argmax(-1)
    pred_label = categories[pred_idx]
    
    return pred_label[0]

In [35]:
categories

Index(['Affection', 'Death', 'Environment', 'Music'], dtype='object')

In [36]:
text1 = '''Tell me, what is it you plan to do
with your one wild and precious life?'''
text1_pred = predicted_label(text1)
text1_pred

1/1 [==============================] - 0s 186ms/step


'Music'

In [37]:
text2 = '''Such aching mystery hides,
in your stardust-glimmer eyes.'''
text2_pred = predicted_label(text2)
text2_pred

1/1 [==============================] - 0s 20ms/step


'Music'

In [38]:
text3 = '''Like space; we are infinite, mysterious, and ever-expanding.
You can look straight into me, and yet still have no clue what hides beyond me.'''
text3_pred = predicted_label(text3)
text3_pred

1/1 [==============================] - 0s 20ms/step


'Affection'

In [39]:
text4 = '''Music, when soft voices die,
Vibrates in the memory'''
text4_pred = predicted_label(text4)
text4_pred

1/1 [==============================] - 0s 20ms/step


'Death'

In [40]:
text5 = '''If nothing saves us from death, at least love should save us from life'''
text5_pred = predicted_label(text5)
text5_pred

1/1 [==============================] - 0s 21ms/step


'Death'